In [1]:
import pandas as pd
import numpy as np
import urllib.request, json 
import re
import pprint

In [2]:
pp = pprint.PrettyPrinter(depth=6)

In [47]:
modules = []
with urllib.request.urlopen("http://api.nusmods.com/2017-2018/1/moduleCodes.json") as url:
    modules = json.loads(url.read().decode())
    
modules = np.array(modules)


with urllib.request.urlopen("http://api.nusmods.com/2017-2018/2/moduleCodes.json") as url:
    modules_2 = json.loads(url.read().decode())
    
modules_2 = np.array(modules_2)

print(modules)
print(modules_2)
print('DSC5101' in modules)

['ACC1002' 'ACC1002X' 'ACC1006' ..., 'UTC3102' 'ZB3310' 'ZB3312']
['ACC1002' 'ACC1002X' 'ACC1006' ..., 'ZB3288' 'ZB3289' 'ZB4199']
True


In [42]:
# only take in the mods with these prefixes
# prefix = ["FIN", "DSC", "MNO", "MKT", "BSP", "ACC", "HR", "TR"]
modules_filtered = np.array(list(filter(lambda module: bool(re.compile("^(FIN|DSC|MNO|MKT|BSP|ACC|HR|TR)",).match(module)), modules)))

# remove "X" from suffix
modules_filtered = np.array(list(filter(lambda module: module[-1] != 'X', modules_filtered)))

modules_filtered_2 = np.array(list(filter(lambda module: bool(re.compile("^(FIN|DSC|MNO|MKT|BSP|ACC|HR|TR)",).match(module)), modules_2)))

# remove "X" from suffix
modules_filtered_2 = np.array(list(filter(lambda module: module[-1] != 'X', modules_filtered_2)))
modules_filtered_2

array(['ACC1002', 'ACC1006', 'ACC1701', 'ACC2002', 'ACC3601', 'ACC3603',
       'ACC3604', 'ACC3605', 'ACC3606', 'ACC3613', 'ACC3614', 'ACC3616',
       'ACC3619', 'ACC4611', 'ACC4614', 'ACC5001', 'BSP1004', 'BSP1005',
       'BSP2001', 'BSP2005', 'BSP3001A', 'BSP3001B', 'BSP3516', 'DSC1007',
       'DSC2006', 'DSC2008', 'DSC3201', 'DSC3202', 'DSC3203', 'DSC3214',
       'DSC3215', 'DSC3216', 'DSC3224', 'DSC4211C', 'DSC4211G', 'DSC4213',
       'DSC4215', 'DSC4217', 'DSC5102', 'DSC5121', 'DSC5211A', 'DSC5211B',
       'DSC5211C', 'FIN2004', 'FIN3101A', 'FIN3101B', 'FIN3102', 'FIN3103',
       'FIN3103B', 'FIN3113', 'FIN3117', 'FIN3118', 'FIN3120B', 'FIN3120C',
       'FIN3120D', 'FIN3130', 'FIN3131', 'FIN4112G', 'FIN4112K',
       'FIN4113A', 'FIN4113B', 'FIN4117', 'FIN4122', 'FIN4123', 'FIN6003',
       'MKT1003', 'MKT1705', 'MKT2401A', 'MKT2401B', 'MKT2413', 'MKT3402A',
       'MKT3402B', 'MKT3402C', 'MKT3412', 'MKT3415', 'MKT3417', 'MKT3418',
       'MKT3420', 'MKT3421', 'MKT4413', 

In [39]:
# define a function that gives us the json from the thing

def get_json(module_code):
    with urllib.request.urlopen("http://api.nusmods.com/2017-2018/1/modules/" + module_code + ".json") as url:
        data = json.loads(url.read().decode())
        
        # remove CorsBiddingStats
        try:
            del data['CorsBiddingStats']
        except:
            pass
        return data
    
    
def get_json_2(module_code):
    with urllib.request.urlopen("http://api.nusmods.com/2017-2018/2/modules/" + module_code + ".json") as url:
        data = json.loads(url.read().decode())
        
        # remove CorsBiddingStats
        try:
            del data['CorsBiddingStats']
        except:
            pass
        return data

pp.pprint((get_json('FIN2004')))
# pp.pprint(get_json('DSC2004'))

{'Department': 'Finance',
 'ExamDate': '2017-11-27T17:00+0800',
 'LecturePeriods': ['Tuesday Afternoon',
                    'Monday Morning',
                    'Monday Afternoon',
                    'Tuesday Morning',
                    'Wednesday Morning',
                    'Thursday Morning',
                    'Wednesday Afternoon',
                    'Friday Morning',
                    'Thursday Afternoon',
                    'Friday Afternoon'],
 'ModuleCode': 'FIN2004',
 'ModuleCredit': '4',
 'ModuleDescription': 'This course helps students to understand the key '
                      'concepts and tools in Finance. It provides a broad '
                      'overview of the financial environment under which a '
                      'firm operates. It equips the students with the '
                      'conceptual and analytical skills necessary to make '
                      'sound financial decisions for a firm. Topics to be '
                      'covered inc

In [24]:
headers = []
timeslots = ['0800', '0900', '1000', '1100', '1200', '1300', '1400', '1500', '1600', '1700', '1800', '1900', '2000', '2100', '2200']
days = ['mon', 'tue', 'wed', 'thu', 'fri']

# module columns
headers.extend(['module', 'name', 'sectional', 'module_sectional'])

# timeslots
for day in days:
    for timeslot in timeslots:
        headers.extend([day + timeslot])
        
def build_df(data):
    return pd.DataFrame(data, columns=headers)

x = []
for i in range(0, 79):
    x.append(i)
    
    
build_df([x])

,module,name,sectional,module_sectional,mon0800,mon0900,mon1000,mon1100,mon1200,mon1300,...,fri1300,fri1400,fri1500,fri1600,fri1700,fri1800,fri1900,fri2000,fri2100,fri2200
0,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78


In [25]:
def convert_time_info_into_list(day, start, end):
    day = day.lower()[:3]
    started = False
    ended = False
    
    timelist = []
    
    for currday in days:
        for slot in timeslots:
            if currday+slot == day+start:
                started = True
            if currday+slot == day+end:
                ended = True
            if (started & ~ended):
                timelist.append(1)
            else:
                timelist.append(0)

    return timelist

convert_time_info_into_list('Monday', '1000', '1200')


dsc3214 = ['DSC3214', 'Optimisation', 'A', 'DSC3214_A']
dsc3214.extend(convert_time_info_into_list('Friday', '1400', '1700'))
build_df([dsc3214])
    

,module,name,sectional,module_sectional,mon0800,mon0900,mon1000,mon1100,mon1200,mon1300,...,fri1300,fri1400,fri1500,fri1600,fri1700,fri1800,fri1900,fri2000,fri2100,fri2200
0,DSC3214,Optimisation,A,DSC3214_A,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0


In [34]:
def convert_mod_info_into_lists(json):
    # this will create a LIST OF LIST, where each inner list is 1 module_sectional!!
    # [modulecode, sectional, mod_sectional, name, [day, start, end]]
    module = json['ModuleCode']
    title = json['ModuleTitle']
    try:
        timeslots = json['Timetable']
    except:
        print('Timetable not found for', module)
        timeslots = []
    
    mylist = []
    
    for slot in timeslots:
        sect = slot['ClassNo']
        start = slot['StartTime']
        end = slot['EndTime']
        day = slot['DayText']
        mylist.append([[module, title, sect, module+sect], (day, start, end)])
    
    return mylist

fin2004 = convert_mod_info_into_lists(get_json('FIN2004'))
fin2004

[[['FIN2004', 'Finance', 'J01', 'FIN2004J01'], ('Tuesday', '1400', '1700')],
 [['FIN2004', 'Finance', 'J02', 'FIN2004J02'], ('Monday', '0800', '1100')],
 [['FIN2004', 'Finance', 'J03', 'FIN2004J03'], ('Monday', '1400', '1700')],
 [['FIN2004', 'Finance', 'J04', 'FIN2004J04'], ('Tuesday', '0800', '1100')],
 [['FIN2004', 'Finance', 'J05', 'FIN2004J05'], ('Tuesday', '1400', '1700')],
 [['FIN2004', 'Finance', 'J06', 'FIN2004J06'], ('Wednesday', '0800', '1100')],
 [['FIN2004', 'Finance', 'J07', 'FIN2004J07'], ('Thursday', '0800', '1100')],
 [['FIN2004', 'Finance', 'J08', 'FIN2004J08'], ('Wednesday', '1200', '1500')],
 [['FIN2004', 'Finance', 'J09', 'FIN2004J09'], ('Friday', '0800', '1100')],
 [['FIN2004', 'Finance', 'J10', 'FIN2004J10'], ('Thursday', '1400', '1700')],
 [['FIN2004', 'Finance', 'J11', 'FIN2004J11'], ('Friday', '1400', '1700')]]

In [36]:
def convert_modlist_into_df(modlist):
    newlist = []
    for mod in modlist:
#         print(mod[1])
        mod_time = convert_time_info_into_list(mod[1][0], mod[1][1], mod[1][2])
        newmod = []
        newmod.extend(mod[0])
        newmod.extend(mod_time)
        newlist.append(newmod)
        
    return newlist

build_df(convert_modlist_into_df(fin2004))

,module,name,sectional,module_sectional,mon0800,mon0900,mon1000,mon1100,mon1200,mon1300,...,fri1300,fri1400,fri1500,fri1600,fri1700,fri1800,fri1900,fri2000,fri2100,fri2200
0,FIN2004,Finance,J01,FIN2004J01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,FIN2004,Finance,J02,FIN2004J02,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,FIN2004,Finance,J03,FIN2004J03,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,FIN2004,Finance,J04,FIN2004J04,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,FIN2004,Finance,J05,FIN2004J05,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,FIN2004,Finance,J06,FIN2004J06,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,FIN2004,Finance,J07,FIN2004J07,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,FIN2004,Finance,J08,FIN2004J08,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,FIN2004,Finance,J09,FIN2004J09,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,FIN2004,Finance,J10,FIN2004J10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
mod_sect_list = []
for mod in modules_filtered:
    mod_json = get_json(mod)
    mod_sect_list.extend(convert_mod_info_into_lists(mod_json))
    
df = build_df(convert_modlist_into_df(mod_sect_list))

Timetable not found for ACC4619
Timetable not found for ACC4629
Timetable not found for DSC3229
Timetable not found for DSC4219
Timetable not found for FIN3129
Timetable not found for FIN4119
Timetable not found for FIN4129
Timetable not found for MKT3429
Timetable not found for MKT4419
Timetable not found for MKT4429
Timetable not found for MNO3329
Timetable not found for MNO4319
Timetable not found for TR3002N
Timetable not found for TR3202
Timetable not found for TR3203E
Timetable not found for TR3203N
Timetable not found for TR3203P
Timetable not found for TR3203T
Timetable not found for DSC3222M
Timetable not found for DSC3239
Timetable not found for DSC4229
Timetable not found for FIN3139
Timetable not found for MKT3439
Timetable not found for MNO4329
('Thursday', '0900', '1100')
('Wednesday', '0900', '1100')
('Wednesday', '1200', '1400')
('Monday', '0900', '1000')
('Monday', '1000', '1100')
('Monday', '1200', '1300')
('Monday', '1300', '1400')
('Friday', '0900', '1000')
('Friday

In [38]:
df.to_csv('1718_s1_modsect_timeslots.csv')

In [48]:
mod_sect_list_2 = []
for mod in modules_filtered_2:
    mod_json = get_json_2(mod)
    mod_sect_list.extend(convert_mod_info_into_lists(mod_json))
    
df = build_df(convert_modlist_into_df(mod_sect_list))

Timetable not found for ACC1002
Timetable not found for ACC1006
Timetable not found for ACC1701
Timetable not found for ACC2002
Timetable not found for ACC3601
Timetable not found for ACC3603
Timetable not found for ACC3604
Timetable not found for ACC3605
Timetable not found for ACC3606
Timetable not found for ACC3613
Timetable not found for ACC3614
Timetable not found for ACC3616
Timetable not found for ACC3619
Timetable not found for ACC4611
Timetable not found for ACC4614
Timetable not found for ACC5001
Timetable not found for BSP1004
Timetable not found for BSP1005
Timetable not found for BSP2001
Timetable not found for BSP2005
Timetable not found for BSP3001A
Timetable not found for BSP3001B
Timetable not found for BSP3516
Timetable not found for DSC1007
Timetable not found for DSC2006
Timetable not found for DSC2008
Timetable not found for DSC3201
Timetable not found for DSC3202
Timetable not found for DSC3203
Timetable not found for DSC3214
Timetable not found for DSC3215
Timeta

In [50]:
df.to_csv('1718_s2_modsect_timeslots.csv')

<br /><br /><br /><br /><br />
## Generating a Prereq/Preclu list

In [51]:
def convert_mod_into_prereq_preclu_list(json):
    # this will create a LIST OF LIST, where each inner list is 1 module_sectional!!
    # [modulecode, sectional, mod_sectional, name, [day, start, end]]
    module = json['ModuleCode']
    title = json['ModuleTitle']
    try:
        prereq = json['Prerequisite']
    except:
        prereq = ''
    try:
        preclu = json['Preclusion']
    except:
        preclu = ''
    
    return [module, title, prereq, preclu]

fin2004 = convert_mod_into_prereq_preclu_list(get_json('FIN2004'))
fin2004

['FIN2004',
 'Finance',
 'Students must have completed BK1003 or BZ1002 or BH1002 or FNA1002/ACC1002 or FNA1002X/ACC1002X or FNA1002E or BH1002E or EC3212 or EG1422 before they are allowed to take FIN2004.',
 'Students who have taken CS2251 or EC3209 or EC3333 or BK2004 or BZ2004 or BH2004 or FNA2004 are not allowed to take FIN2004. 1st Year BSc(PFM), all BSc (RE) and Computational Finance are not allowed to take FIN2004.']

In [52]:
prereq_preclu_list = []
for mod in modules_filtered:
    try:
        myjson = get_json(mod)
    except:
        myjson = {}
        myjson['ModuleCode'] = mod
        myjson['ModuleTitle'] = 'NOT FOUND'
    prereq_preclu_list.append(convert_mod_into_prereq_preclu_list(myjson))
    
prereq_df = pd.DataFrame(prereq_preclu_list, columns=['Module', 'Title', 'Prereq', 'Preclu'])
prereq_df

,Module,Title,Prereq,Preclu
0,ACC1002,Financial Accounting,,Students who have passed FNA1002 are not allow...
1,ACC1006,Accounting Information Systems,FNA1002 or ACC1002,Students who have passed FNA1006 are not allow...
2,ACC1701,Accounting for Decision Makers,,ACC1002; ACC1002X
3,ACC2002,Managerial Accounting,Students must have completed BK1003 or BZ1002 ...,BH2002 or BZ3102 or BK2001 or FNA2002 or IE4242
4,ACC3601,Corporate Accounting and Reporting,FNA1002 or ACC1002,BH3111 or BZ3101 or BK3106 or FNA3111
5,ACC3602,Managerial Planning and Control,FNA2002 or ACC2002,Students who have passed FNA3112 are not allow...
6,ACC3603,Assurance and Attestation,FNA1002/ACC1002 and FNA2002 (Students who are ...,Students who have passed FNA3121 are not allow...
7,ACC3604,Corporate and Securities Law,BSP1004,Students who have passed FNA3122 or LL4055 are...
8,ACC3605,Taxation,FNA1002/ACC1002 and BSP1004,Students who have passed FNA3127 or LL4056 are...
9,ACC3606,Advanced Corporate Accounting and Reporting,FNA3111 or ACC3601,Students who have passed FNA3123 are not allow...


In [53]:
prereq_df.to_csv('prereq_preclu.csv')

In [54]:
prereq_preclu_list = []
for mod in modules_filtered_2:
    try:
        myjson = get_json_2(mod)
    except:
        myjson = {}
        myjson['ModuleCode'] = mod
        myjson['ModuleTitle'] = 'NOT FOUND'
    prereq_preclu_list.append(convert_mod_into_prereq_preclu_list(myjson))
    
prereq_df = pd.DataFrame(prereq_preclu_list, columns=['Module', 'Title', 'Prereq', 'Preclu'])
prereq_df

,Module,Title,Prereq,Preclu
0,ACC1002,Financial Accounting,,Students who have passed FNA1002 are not allow...
1,ACC1006,Accounting Information Systems,FNA1002 or ACC1002,Students who have passed FNA1006 are not allow...
2,ACC1701,Accounting for Decision Makers,,ACC1002; ACC1002X
3,ACC2002,Managerial Accounting,Students must have completed BK1003 or BZ1002 ...,BH2002 or BZ3102 or BK2001 or FNA2002 or IE4242
4,ACC3601,Corporate Accounting and Reporting,FNA1002 or ACC1002,BH3111 or BZ3101 or BK3106 or FNA3111
5,ACC3603,Assurance and Attestation,FNA1002/ACC1002 and FNA2002 (Students who are ...,Students who have passed FNA3121 are not allow...
6,ACC3604,Corporate and Securities Law,BSP1004,Students who have passed FNA3122 or LL4055 are...
7,ACC3605,Taxation,FNA1002/ACC1002 and BSP1004,Students who have passed FNA3127 or LL4056 are...
8,ACC3606,Advanced Corporate Accounting and Reporting,FNA3111 or ACC3601,Students who have passed FNA3123 are not allow...
9,ACC3613,Advanced Assurance and Attestation,FNA3121 or ACC3603,Students who have passed FNA3128 are not allow...


In [55]:
prereq_df.to_csv('prereq_preclu_2.csv')